In [ ]:
import time
from enum import Enum
import RPi.GPIO as GPIO

This notebook is basic tutorial for L298N H-bridge ICs, including:



## 1. L298 Pinout

<img src="Figures/Introduction-to-L298_5.png.webp" alt="L298 Pinout" width="500"/>

The L298 is able to drive two motors, denoted as motor A and motor B. The following pins are used for motor A ("-->" shows how they should be wired up.): 
- Pin 5: INPUT 1 --> Rpi GPIO
- Pin 7: INPUT 2 --> Rpi GPIO
- Pin 2: OUTPUT 1 --> Motor A power
- Pin 3: OUTPUT 2 --> Motor A power
- Pin 6: ENABLE A --> Rpi GPIO
- Pin 1: CURRENT SENSING A --> Ground (This pin is used to measure how much current the motor is drawing, we don't demonstrate this in this tutorial.)

For motor B:
- Pin 10: INPUT 3 --> Rpi GPIO
- Pin 12: INPUT 4 --> Rpi GPIO
- Pin 13: OUTPUT 3 --> Motor B power
- Pin 14: OUTPUT 4 --> Motor B power
- Pin 11: ENABLE B --> Rpi GPIO
- Pin 15: CURRENT SENSING B --> Ground (This pin is used to measure how much current the motor is drawing, we don't demonstrate this in this tutorial.)

The other pins are common for both motors:
- Pin 9: LOGIC SUPPLY VOLTAGE Vss --> 5V
- Pin 8: GND --> Ground
- Pin 4: SUPPLY VOLTAGE Vs --> 12V

## 2. Assigning GPIO

In [ ]:
motor_L_in1 = 18
motor_L_in2 = 23
motor_L_en = 13

motor_R_in2 = 24
motor_R_in1 = 25
motor_R_en = 19

GPIO.setmode(GPIO.BCM)
GPIO.setup(motor_L_in1, GPIO.OUT)
GPIO.setup(motor_L_in2, GPIO.OUT)
GPIO.setup(motor_R_in1, GPIO.OUT)
GPIO.setup(motor_R_in2, GPIO.OUT)
GPIO.setup(motor_L_en, GPIO.OUT)
GPIO.setup(motor_R_en, GPIO.OUT)